In [2]:
from abc import ABC, abstractmethod
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
from abc import ABC, abstractmethod
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Data Ingestion via Factory").getOrCreate()

# Interface de Produto
class DataIngestion(ABC):
    @abstractmethod
    def ingest(self, source):
        pass

# Produto Concreto para CSV
class CSVIngestion(DataIngestion):
    def __init__(self, line_delimiter="\n"):
        self.line_delimiter = line_delimiter

    def ingest(self, source):
        return (
            spark.read
            .option("header", "true")
            .option("delimiter", self.line_delimiter)
            .csv(source)
        )

# Produto Concreto para Parquet
class ParquetIngestion(DataIngestion):
    def ingest(self, source):
        return spark.read.parquet(source)

# Creator Base
class ReadData(ABC):
    @abstractmethod
    def factory_method(self) -> DataIngestion:
        pass

    def run_ingestion(self, source):
        ingestion = self.factory_method()
        df = ingestion.ingest(source)
        return df

# Creator Concreto para CSV
class CSVReader(ReadData):
    def __init__(self, line_delimiter="\n"):
        self.line_delimiter = line_delimiter

    def factory_method(self) -> DataIngestion:
        return CSVIngestion(self.line_delimiter)

# Creator Concreto para Parquet
class ParquetReader(ReadData):
    def factory_method(self) -> DataIngestion:
        return ParquetIngestion()

# Código do cliente
def client_code(reader: ReadData, source: str):
    df = reader.run_ingestion(source)
    return df
